In [2]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [12]:
trainfile = "train.csv"
testfile = "test.csv"

data = pd.read_csv(trainfile)
testdata = pd.read_csv(testfile)
names = data['smiles']
testnames = testdata['smiles']
testId = testdata['Id']
data = data.drop('smiles',1)
test = testdata.drop(['smiles','Id'],1)
ytrain = data['gap']
xtrain = data.drop('gap',1)

In [11]:
test.head

<bound method DataFrame.head of             Id  feat_001  feat_002  feat_003  feat_004  feat_005  feat_006  \
0            1         0         0         0         0         1         1   
1            2         0         0         0         0         1         1   
2            3         1         0         0         0         1         1   
3            4         1         0         0         0         1         1   
4            5         0         0         0         0         1         0   
5            6         1         0         0         0         1         0   
6            7         1         0         0         0         1         1   
7            8         1         0         0         0         1         0   
8            9         0         0         0         0         1         0   
9           10         0         0         0         0         1         1   
10          11         1         0         0         0         1         1   
11          12         1        

In [13]:
rand_for = sklearn.ensemble.RandomForestRegressor()
rand_for.fit(xtrain,ytrain)
preds_rf = rand_for.predict(test)

In [14]:
xgtrain = xgb.DMatrix(xtrain, label=ytrain)
xgtest = xgb.DMatrix(test)
param = {'max_depth':100, 'eta':1, 'silent':0, 'objective':'reg:linear','learning_rate':0.6}
num_round = 10
xg = xgb.train(param, xgtrain, num_round)
preds_xg = xg.predict(xgtest)

In [15]:
avg_pred = (preds_rf+preds_xg)/2

In [16]:
avg_pred

array([ 1.46628764,  1.66040535,  1.55036319, ...,  1.46628764,
        1.50621206,  1.82644941])

In [39]:
result = pd.DataFrame(testId)
result['Prediction'] = avg_pred


In [40]:
result.to_csv('predictions.csv', index=None)

In [ ]:
mols = []
for name in names.tolist():
    m = Chem.MolFromSmiles(name)
    mols.append(m)